<a href="https://colab.research.google.com/github/Growth-Hackers-SNU/sandbox-youtube-anlaysis/blob/main/code/deeplevel/DeepModelMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make Pre trained Deep Models

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm, tqdm_notebook
import pickle 

import matplotlib.pyplot as plt

In [ ]:
! pip install soynlp
! pip install kiwipiepy

     |████████████████████████████████| 419kB 4.1MB/s 
     |████████████████████████████████| 3.7MB 4.3MB/s 
     |████████████████████████████████| 57.3MB 96kB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.9.0-cp37-none-any.whl size=57496291 sha256=e46576c4c6edb87e1693c7c99fceca8b4b7cd11e856dbbb60dd94bf471e7b5a5
  Stored in directory: /root/.cache/pip/wheels/61/19/a2/f85a7031e0ff829231b4562e6460a84fef599080bbb2573ef7
Successfully built kiwipiepy-model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set Training Data

In [ ]:
Youtube_comment_label = pd.read_csv("/content/drive/Shareddrives/[GH x Sandbox]/최종분류/sample_final.csv", index_col = 1)
Youtube_comment_label = Youtube_comment_label[['comment', 'class']]
# 7번 클래스 -> 0으로 처리 : 나중에 편합니다.
Youtube_comment_label.loc[Youtube_comment_label['class'] ==  7, 'class'] = 0
Youtube_comment_label.head()

,comment,class
index,,
1,흠.. 일단 전 안맞을래요,0
2,자식새끼 키워봤자 아무소용업다..,0
3,방테나ㅋㅋㅋ,2
4,하람님 저고민있어욧!!\n노래잘부르고싶은데..어떻게하면 잘 부를수있을까요?오늘도 영...,1
5,희재도 준비하고있고 발표예정이예요\n다시 확인해주세요 모두 준비하고있어요 계약때문에...,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# 각 라벨 갯수가 균형된 데이터 셋, 그냥 raw한 데이터 셋 두 개씩 클래스 3개, 7개 총 4가지 데이터 셋을 만들 겁니다.
Youtube_comment_label3 = Youtube_comment_label.copy()
Youtube_comment_label7 = Youtube_comment_label.copy()

# Youtube_comment_label3
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  1, 'class'] = 1
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  2, 'class'] = 1
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  3, 'class'] = 1
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  4, 'class'] = 2
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  5, 'class'] = 2
Youtube_comment_label3.loc[Youtube_comment_label3['class'] ==  6, 'class'] = 2
print(Youtube_comment_label3.head())
print(f"총 데이터 갯수 : {len(Youtube_comment_label3['comment'])}")
print("-------------------------------------------------------------------------------------------")

# Youtube_comment_label7
# 더 건드릴 것이 없다
print(Youtube_comment_label7.head())
print(f"총 데이터 갯수 : {len(Youtube_comment_label7['comment'])}")
print("-------------------------------------------------------------------------------------------")

                                                 comment  class
index                                                          
1                                         흠.. 일단 전 안맞을래요      0
2                                     자식새끼 키워봤자 아무소용업다..      0
3                                                 방테나ㅋㅋㅋ      1
4      하람님 저고민있어욧!!\n노래잘부르고싶은데..어떻게하면 잘 부를수있을까요?오늘도 영...      1
5      희재도 준비하고있고 발표예정이예요\n다시 확인해주세요 모두 준비하고있어요 계약때문에...      0
총 데이터 갯수 : 22000
-------------------------------------------------------------------------------------------
                                                 comment  class
index                                                          
1                                         흠.. 일단 전 안맞을래요      0
2                                     자식새끼 키워봤자 아무소용업다..      0
3                                                 방테나ㅋㅋㅋ      2
4      하람님 저고민있어욧!!\n노래잘부르고싶은데..어떻게하면 잘 부를수있을까요?오늘도 영...      1
5      희재도 준비하고있고 발표예정이예요\n다시 확인해주세요 모두 준비하고있어요 계약때문에...   

In [ ]:
train_data, test_data = train_test_split(Youtube_comment_label7, test_size=1000, random_state=7607)

In [ ]:
X_train = train_data[['comment']]
X_test = test_data[['comment']]
y_train = train_data['class']
y_test = test_data['class']

## JAMO

### Tokenization

In [ ]:
# 타임 스탬프 삭제
p = re.compile('\d+:\d+')

def delete_time_stamp(df):
    for ix, row in tqdm_notebook(df.iterrows()):
        time_stamp_list = p.findall(str(row['comment']))
        new_comment = str(row['comment'])
        for time_stamp in time_stamp_list:
            new_comment = new_comment.replace(time_stamp, '')
        row['comment'] = new_comment.lstrip()
    return df

In [ ]:
delete_time_stamp(X_train)
delete_time_stamp(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


,comment
index,
6068,와 내집마련 축하드려요!!
18511,마리:왜 또 코피가!?이제 잘건데\n거머리:배불리 먹었다\n마리:으악!거머리가 내코...
20366,성악전공 학생임둥🙋🏻‍♀️ 함 부터보자요❣️
11462,나이서
12892,"김나영, 신우, 이준, 지수씨의 노필터 TV 최고!\n노필터티비 때문에 밤에 잠을 ..."
...,...
2658,피티가 안나온 이유가 설마.. 조..폭...?
20008,실력파라는게\n킬포인트죠????
9957,너무 귀여워용..어깨 피고 나가시는 모습


In [ ]:
from soynlp.hangle import compose, decompose, character_is_korean 
doublespace_pattern = re.compile('\s+') 

def jamo_sentence(sent): 
    def transform(char): 
        if char == ' ': 
            return char 
        cjj = decompose(char) 
        if len(cjj) == 1: 
            return cjj 
        cjj_ = ''.join(c if c != ' ' else '_' for c in cjj) 
        return cjj_ 
    
    sent_ = [] 
    for char in sent: 
        if character_is_korean(char): 
            sent_.append(transform(char)) 
        else: 
            sent_.append(char) 
    sent_ = doublespace_pattern.sub(' ', ''.join(sent_)) 
    return sent_ 
        
def jamo_to_word(jamo): 
    jamo_list, idx = [], 0 
    while idx < len(jamo): 
        if not character_is_korean(jamo[idx]): 
            jamo_list.append(jamo[idx]) 
            idx += 1 
        else: 
            jamo_list.append(jamo[idx:idx + 3]) 
            idx += 3 
            
    word = "" 
    for jamo_char in jamo_list: 
        if len(jamo_char) == 1: 
            word += jamo_char 
        elif jamo_char[2] == "_": 
            word += compose(jamo_char[0], jamo_char[1], " ") 
        else: 
            word += compose(jamo_char[0], jamo_char[1], jamo_char[2]) 
            
    return word

# 출처: https://joyhong.tistory.com/137 [옳은 길로..]

In [ ]:
X_train = X_train.apply(lambda row : jamo_sentence(str(row['comment'])).strip().split(" "), axis=1)
X_test = X_test.apply(lambda row : jamo_sentence(str(row['comment'])).strip().split(" "), axis=1)

### Load FastText JAMO

In [ ]:
from gensim.models import KeyedVectors, FastText

In [ ]:
model_fname = "/content/drive/Shareddrives/[GH x Sandbox]/code/DeepLevel/fasttext/jamo_fasttext"
embedding_model = FastText.load(model_fname)

### Encoding

In [ ]:
def sen_to_seq(s_list): # s = ['ㅇㅣ_ㅎㅛ_ㅂㅐ_', 'ㅂㅏ_ㅂㅗ_']
    seq = list()
    for s in s_list:
        try:
            seq_value = embedding_model.wv.index2word.index(s)+1
        except:
            seq_value = 0 # 없으면 0
        seq.append(seq_value)
    return seq

def seq_padding(seq, max_len):
    if len(seq) < max_len:
        n = max_len - len(seq)
        zero_list = [0] * n
        seq += zero_list
    elif len(seq) > max_len:
        seq = seq[:max_len] 
    
    return seq   

In [ ]:
max_len = max(len(l) for l in X_train)
len_distribution = [len(l) for l in X_train]
np.mean(len_distribution)
sentence_len = 15

In [ ]:
word_seq_train = X_train.apply(lambda row : sen_to_seq(row))
word_seq_test = X_test.apply(lambda row : sen_to_seq(row))

In [ ]:
word_seq_train = word_seq_train.apply(lambda row : seq_padding(row, sentence_len))
word_seq_test = word_seq_test.apply(lambda row : seq_padding(row, sentence_len))
word_seq_train = np.array(word_seq_train.to_list())
word_seq_test = np.array(word_seq_test.to_list())

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_catg = to_categorical(y_train, num_classes=3)
y_test_catg = to_categorical(y_test, num_classes=3)

### Build Embedding Matrix

In [ ]:
embedding_matrix = np.zeros((embedding_model.wv.vectors.shape[0]+1, embedding_model.wv.vectors.shape[1]))
np.shape(embedding_matrix)

(283368, 100)

In [ ]:
vocab_size = np.shape(embedding_matrix)[0]
vocab_size

283368

In [ ]:
for i in tqdm(range(len(embedding_model.wv.vectors))):
    embedding_matrix[i+1] = embedding_model.wv.vectors[i]

100%|██████████| 283367/283367 [00:00<00:00, 829255.65it/s]


### Modeling: LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
lstm_model = Sequential()

e = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=sentence_len, trainable=False) # input_length = sentence_len
lstm_model.add(e)
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32, return_sequences=False))
# lstm_model.add(BatchNormalization())
lstm_model.add(Dense(16, activation = 'tanh'))
lstm_model.add(Dense(3, activation = 'softmax')) # 여기서 3은 최종 라벨 갯수

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 100)           28336800  
_________________________________________________________________
lstm (LSTM)                  (None, 15, 64)            42240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 28,392,035
Trainable params: 55,235
Non-trainable params: 28,336,800
_________________________________________________________________


In [ ]:
checkpoint_path = './checkpoints/my_checkpoint'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=False,
                             save_best_only=True,
                             monitor='val_acc',
                             verbose=1)

lstm_model.fit(word_seq_train, y_train_catg, 
               epochs=30, verbose=2, 
               validation_data=(word_seq_test, y_test_catg),
               callbacks=[checkpoint])

Epoch 1/30
657/657 - 8s - loss: 0.7787 - acc: 0.6643 - val_loss: 0.7864 - val_acc: 0.6540

Epoch 00001: val_acc improved from -inf to 0.65400, saving model to ./checkpoints/my_checkpoint


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint/assets


Epoch 2/30
657/657 - 3s - loss: 0.7342 - acc: 0.6844 - val_loss: 0.7917 - val_acc: 0.6480

Epoch 00002: val_acc did not improve from 0.65400
Epoch 3/30
657/657 - 3s - loss: 0.7192 - acc: 0.6909 - val_loss: 0.7798 - val_acc: 0.6590

Epoch 00003: val_acc improved from 0.65400 to 0.65900, saving model to ./checkpoints/my_checkpoint


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint/assets


Epoch 4/30
657/657 - 3s - loss: 0.7059 - acc: 0.6978 - val_loss: 0.7894 - val_acc: 0.6540

Epoch 00004: val_acc did not improve from 0.65900
Epoch 5/30
657/657 - 3s - loss: 0.6895 - acc: 0.7068 - val_loss: 0.7936 - val_acc: 0.6590

Epoch 00005: val_acc did not improve from 0.65900
Epoch 6/30
657/657 - 3s - loss: 0.6722 - acc: 0.7139 - val_loss: 0.8129 - val_acc: 0.6490

Epoch 00006: val_acc did not improve from 0.65900
Epoch 7/30
657/657 - 3s - loss: 0.6534 - acc: 0.7227 - val_loss: 0.7964 - val_acc: 0.6550

Epoch 00007: val_acc did not improve from 0.65900
Epoch 8/30
657/657 - 3s - loss: 0.6331 - acc: 0.7310 - val_loss: 0.8321 - val_acc: 0.6430

Epoch 00008: val_acc did not improve from 0.65900
Epoch 9/30
657/657 - 3s - loss: 0.6073 - acc: 0.7438 - val_loss: 0.8504 - val_acc: 0.6350

Epoch 00009: val_acc did not improve from 0.65900
Epoch 10/30
657/657 - 3s - loss: 0.5775 - acc: 0.7563 - val_loss: 0.8947 - val_acc: 0.6310

Epoch 00010: val_acc did not improve from 0.65900
Epoch 11/30


In [ ]:
# save best model
lstm_model.load_weights(checkpoint_path)
saved_model_path = '/content/drive/Shareddrives/[GH x Sandbox]/code/DeepLevel/model/lstm_jamo3'
lstm_model.save(saved_model_path, save_format='h5')

## EUMJUL

### Tokenization

In [ ]:
X_train = train_data[['comment']]
X_test = test_data[['comment']]
y_train = train_data['class']
y_test = test_data['class']

In [ ]:
def eumjul_sentence(texts):
    tokenized = []
    for j in range(len(texts)):
        tokenized.append(texts[j])
    return tokenized

In [ ]:
X_train = X_train.apply(lambda row : eumjul_sentence(str(row['comment'])), axis=1)
X_test = X_test.apply(lambda row : eumjul_sentence(str(row['comment'])), axis=1)

### Load FastText EUMJUL

In [ ]:
model_fname = "/content/drive/Shareddrives/[GH x Sandbox]/code/DeepLevel/fasttext/fasttext_eumjul"
embedding_model = FastText.load(model_fname)

In [ ]:
def sen_to_seq(s_list): # s = ['아', '니']
    seq = list()
    for s in s_list:
        try:
            seq_value = embedding_model.wv.index2word.index(s)+1
        except:
            seq_value = 0 # 없으면 0
        seq.append(seq_value)
    return seq

### Encoding

In [ ]:
max_len = max(len(l) for l in X_train)
len_distribution = [len(l) for l in X_train]
np.mean(len_distribution)
sentence_len = 100

In [ ]:
word_seq_train = X_train.apply(lambda row : sen_to_seq(row))
word_seq_test = X_test.apply(lambda row : sen_to_seq(row))

In [ ]:
word_seq_train = word_seq_train.apply(lambda row : seq_padding(row, sentence_len))
word_seq_test = word_seq_test.apply(lambda row : seq_padding(row, sentence_len))
word_seq_train = np.array(word_seq_train.to_list())
word_seq_test = np.array(word_seq_test.to_list())

In [ ]:
y_train_catg = to_categorical(y_train, num_classes=3)
y_test_catg = to_categorical(y_test, num_classes=3)

IndexError: ignored

### Build Embedding Matrix

In [ ]:
embedding_matrix = np.zeros((embedding_model.wv.vectors.shape[0]+1, embedding_model.wv.vectors.shape[1]))
np.shape(embedding_matrix)   

In [ ]:
vocab_size = np.shape(embedding_matrix)[0]
vocab_size

5664

In [ ]:
for i in tqdm(range(len(embedding_model.wv.vectors))):
    embedding_matrix[i+1] = embedding_model.wv.vectors[i]

100%|██████████| 5663/5663 [00:00<00:00, 646727.03it/s]


### Modeling: LSTM

In [ ]:
lstm_model = Sequential()

e = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=sentence_len, trainable=False) # input_length = sentence_len
lstm_model.add(e)
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32, return_sequences=False))
# lstm_model.add(BatchNormalization())
lstm_model.add(Dense(16, activation = 'tanh'))
lstm_model.add(Dense(3, activation = 'softmax')) # 여기서 3은 최종 라벨 갯수

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          566400    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 64)           42240     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 51        
Total params: 621,635
Trainable params: 55,235
Non-trainable params: 566,400
_________________________________________________________________


In [ ]:
checkpoint_path = './checkpoints/my_checkpoint2'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=False,
                             save_best_only=True,
                             monitor='val_acc',
                             verbose=1)

lstm_model.fit(word_seq_train, y_train_catg, 
               epochs=30, verbose=2, 
               validation_data=(word_seq_test, y_test_catg),
               callbacks=[checkpoint])

Epoch 1/30
657/657 - 9s - loss: 0.9701 - acc: 0.5310 - val_loss: 0.9804 - val_acc: 0.5260

Epoch 00001: val_acc improved from -inf to 0.52600, saving model to ./checkpoints/my_checkpoint2


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


Epoch 2/30
657/657 - 5s - loss: 0.9608 - acc: 0.5396 - val_loss: 0.9793 - val_acc: 0.5310

Epoch 00002: val_acc improved from 0.52600 to 0.53100, saving model to ./checkpoints/my_checkpoint2


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


...

INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


Epoch 23/30
657/657 - 6s - loss: 0.6768 - acc: 0.7138 - val_loss: 0.7004 - val_acc: 0.7100

Epoch 00023: val_acc did not improve from 0.71700
Epoch 24/30
657/657 - 5s - loss: 0.6723 - acc: 0.7160 - val_loss: 0.6930 - val_acc: 0.7100

Epoch 00024: val_acc did not improve from 0.71700
Epoch 25/30
657/657 - 5s - loss: 0.6651 - acc: 0.7203 - val_loss: 0.7068 - val_acc: 0.7030

Epoch 00025: val_acc did not improve from 0.71700
Epoch 26/30
657/657 - 5s - loss: 0.6606 - acc: 0.7225 - val_loss: 0.6955 - val_acc: 0.7170

Epoch 00026: val_acc did not improve from 0.71700
Epoch 27/30
657/657 - 5s - loss: 0.6536 - acc: 0.7228 - val_loss: 0.7021 - val_acc: 0.7220

Epoch 00027: val_acc improved from 0.71700 to 0.72200, saving model to ./checkpoints/my_checkpoint2


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


INFO:tensorflow:Assets written to: ./checkpoints/my_checkpoint2/assets


Epoch 28/30
657/657 - 5s - loss: 0.6449 - acc: 0.7272 - val_loss: 0.7263 - val_acc: 0.7140

Epoch 00028: val_acc did not improve from 0.72200
Epoch 29/30
657/657 - 5s - loss: 0.6410 - acc: 0.7306 - val_loss: 0.7107 - val_acc: 0.7180

Epoch 00029: val_acc did not improve from 0.72200
Epoch 30/30
657/657 - 5s - loss: 0.6356 - acc: 0.7322 - val_loss: 0.6973 - val_acc: 0.7200

Epoch 00030: val_acc did not improve from 0.72200


In [ ]:
# save best model
lstm_model.load_weights(checkpoint_path)
saved_model_path = '/content/drive/Shareddrives/[GH x Sandbox]/code/DeepLevel/model/lstm_eumjul7'
lstm_model.save(saved_model_path, save_format='h5')